# Exploration des méthodes d'estimation des scores pour les athlètes n'ayant pas participé à la compétition

In [5]:
import sys
import random
from datetime import datetime

from sklearn.decomposition import PCA
from scipy import special
import numpy as np
from numpy.random import randn
from numpy.matlib import repmat
import pandas as pd

from points_methods.tensorial_product_estimator import estimate_tensorial_product
from points_methods.bpca import bpca_fill
from data_handling.database_service import DatabaseService
from points_methods.skill_based_method import get_competitors_from_cursor

## Simulation de données de courses

In [24]:
def make_fake_data(
    nb_competitors, nb_competitions, nb_competitors_per_competition, noise_std=3
):
    competitor_perfs = np.random.normal(100, 10, nb_competitors)
    competition_length = np.random.normal(1, 0.1, nb_competitions)
    scores = competition_length[:, np.newaxis] @ competitor_perfs[
        np.newaxis, :
    ] + noise_std * randn(nb_competitions, nb_competitors)
    print(
        f"Erreur moyenne due au bruit lors de la génération des données : {abs(scores - competition_length[:, np.newaxis] @ competitor_perfs[np.newaxis, :]).mean()}"
    )
    mask = np.zeros_like(scores)
    for i in range(nb_competitions):
        mask[
            i, random.sample(range(nb_competitors), nb_competitors_per_competition)
        ] = 1
    return scores, mask, competitor_perfs, competition_length


def make_fake_data2(
    nb_competitors,
    nb_competitions,
    nb_competitors_per_competition,
    noise_std=3,
    category_coef_std=0.03,
    category_coefs=[1, 1.05, 1.13, 1.2],
    category_part=[0.53, 0.23, 0.16, 0.08],
    verbose=False,
):
    competitor_perfs = np.random.normal(100, 10, nb_competitors)
    boundaries = [0] + [int(nb_competitors * part) for part in np.cumsum(category_part)]
    competition_length = np.random.normal(1, 0.1, nb_competitions)
    scores = np.zeros((nb_competitions, nb_competitors))
    applied_coefs = (
        np.array(category_coefs)[np.newaxis, :]
        + np.random.randn(nb_competitions, len(category_coefs)) * category_coef_std
    )
    for i in range(nb_competitions):
        for j in range(len(category_coefs)):
            scores[i, boundaries[j] : boundaries[j + 1]] = (
                applied_coefs[i, j]
                * competition_length[i]
                * competitor_perfs[boundaries[j] : boundaries[j + 1]]
            )
    scores_with_noise = scores + noise_std * randn(nb_competitions, nb_competitors)
    scores_no_noise = scores
    if verbose:
        print(
            f"Erreur moyenne due au bruit lors de la génération des données : {abs(scores - scores_with_noise).mean()}"
        )
    mask = np.zeros_like(scores)
    for i in range(nb_competitions):
        mask[
            i, random.sample(range(nb_competitors), nb_competitors_per_competition)
        ] = 1
    return scores_with_noise, mask, competitor_perfs, competition_length, scores_no_noise, applied_coefs


In [4]:
s, m, mu, _lambda = make_fake_data(10, 5, 6, noise_std=3)
print(f"mu = {mu}\n")
print(f"lambda = {_lambda}\n")
print(f"Number of competitions per competitor : {np.sum(m, axis=0)}\n")
print(f"scores = \n{s}\n")
print(f"mask = \n{m}\n")

Erreur moyenne due au bruit lors de la génération des données : 2.5522564467497446
mu = [ 86.743991   130.8143107   96.98355928 117.74121436 104.62576778
  95.38920557  88.13345499  99.29917326 106.35231464  72.31162396]

lambda = [1.10393297 1.03859142 0.90858148 1.17503294 0.9970659 ]

Number of competitions per competitor : [4. 4. 1. 1. 5. 3. 4. 4. 1. 3.]

scores = 
[[ 95.13060696 149.8942189  111.47177274 133.56988487 117.93945547
  103.70093232  94.16813052 109.29871214 111.40537881  85.96820849]
 [ 88.31543157 137.81053437  96.37933425 117.37571882 111.67030849
   97.6912931   89.74979685 103.23685082 110.84197142  76.34662016]
 [ 81.35004275 115.13915332  86.16763445 106.65914525  93.7804172
   95.04106828  83.73719036  90.98514355  95.15771104  66.76297312]
 [103.53013905 157.00774103 112.75520909 135.38494369 124.25221024
  115.24029281 105.26120131 121.10370202 126.1046379   83.51712509]
 [ 92.35249083 134.14729694  99.6126872  116.93961163 103.81619248
   89.27355601  86.561

In [5]:
scores, mask, competitor_perfs, competition_length, scores_no_noise, applied_coefs = make_fake_data2(10, 5, 6, noise_std=2, category_coef_std=0.02)

print(f"mu = {competitor_perfs}\n")
print(f"lambda = {competition_length}\n")
print(f"Number of competitions per competitor : {np.sum(mask, axis=0)}\n")
print(f"scores = \n{scores}\n")
print(f"mask = \n{mask}\n")
print(f"applied_coefs = \n{applied_coefs}\n")

Erreur moyenne due au bruit lors de la génération des données : 1.6191904576781875
mu = [ 90.21492419  91.67963743  96.81319464  82.57296873  87.24373442
 108.48316502 111.40716091  95.30862411 101.21983405  98.58167761]

lambda = [0.77590993 0.86151811 0.88153983 1.02450488 0.95946145]

Number of competitions per competitor : [3. 3. 3. 3. 4. 4. 3. 3. 3. 1.]

scores = 
[[ 69.56345949  67.50879978  74.85239614  62.0449582   68.52255169
   88.04619347  90.18236556  81.30608447  85.67196282  92.819453  ]
 [ 77.74797089  81.45816894  84.68324463  73.87088605  78.04811432
   93.33399453 100.82698119  91.36154436  99.90609572  99.92535528]
 [ 75.89941334  81.08323396  79.68463894  69.42010576  76.49834467
  101.75735505 106.47211244  95.18887587 100.95441763 106.78443119]
 [ 89.73778452  97.33466772  99.91492248  83.03858479  91.36245433
  117.90016841 116.50076315 106.57931125 112.63563622 120.67402256]
 [ 87.75877198  82.20307325  91.48007098  80.27686112  84.69396602
  108.5637375  110.06

## Récupération de données réelles

In [7]:
def get_scores_table(starting_date):
    phase = None
    db_service = DatabaseService()
    ending_date = starting_date.replace(year=starting_date.year + 1)
    db_service.get_competition_dates(starting_date)
    competitors_cursor = db_service.get_competitors_on_period(
        starting_date, ending_date, phase=phase
    )
    competitors, nb_participations = get_competitors_from_cursor(competitors_cursor)
    competitions = list(
        db_service.get_competitions_on_period(
            starting_date, ending_date, phase=phase
        )
    )
    scores_df = pd.DataFrame(0, columns=competitors, index=competitions, dtype=float)
    participations_df = pd.DataFrame(0, columns=competitors, index=competitions)
    participations = db_service.get_participations_on_period(
        starting_date, ending_date, phase=phase
    )
    for participation in participations:
        competitor = (
            participation["competitorName"],
            participation["competitorCategory"],
        )
        competition_name = participation["competitionName"]
        score = participation["score"]
        scores_df.at[competition_name, competitor] = score
        participations_df.at[competition_name, competitor] = 1
    return scores_df, participations_df

def filter_score_table(s, m):
    

In [8]:
s, m = get_scores_table(datetime(2019, 1, 1))


In [13]:
print(f"nombre de compétitions : {m.shape[0]}")
print(f"nombre de compétiteurs : {m.shape[1]}")
print(f"nombre moyen de participants par compétition : {m.sum(axis=1).mean()}")

nombre de compétitions : 246
nombre de compétiteurs : 3599
nombre moyen de participants par compétition : 138.369918699187


## Estimation des scores par itérations de PCA

In [6]:
def iter_pca_score_estimation(
    s_masked,
    m,
    n_components=1,
    nb_iter=100,
    initial_noise_std=10,
    noise_std=0,
    verbose=False,
    initialisation="tensorial_product_estimation",
    s=None
):
    # s_masked = s_masked + (
    #     (s_masked.sum() / m.sum())
    #     + np.random.randn(*s_masked.shape) * initial_noise_std
    # ) * (1 - m)
    #s_masked = s_masked + (s_masked.sum() / m.sum()) * (1 - m)
    if initialisation == "tensorial_product_estimation":
        competition_vector, competitor_vector = estimate_tensorial_product(
            pd.DataFrame(s * m), participations_df=pd.DataFrame(m), removing_rate=0
        )
        s_estimated = competition_vector.values[:, np.newaxis] * competitor_vector.values[np.newaxis, :]
        s_masked = s_estimated * (1 - m) + s_masked * m
    elif initialisation == "random":
        s_masked = s_masked + np.random.randn(*s_masked.shape) * initial_noise_std * (1 - m)
    elif initialisation == "zeros":
        pass # s_masked = s_masked * m
    for i in range(nb_iter):
        pca = PCA(n_components=n_components)
        y = pca.fit_transform(s_masked)
        recons_s = pca.inverse_transform(y)
        new_s_masked = (
            recons_s * (1 - m) + s_masked * m + np.random.randn(*s_masked.shape) * noise_std * (1 - m)
        )
        old_s_masked = s_masked
        s_masked = new_s_masked
        if i % max(nb_iter//10, 1) == 0 and verbose:
            print(
                f"\niteration {i}, écart absolu moyen sur la reconstruction des données connues = {(abs(pca.inverse_transform(y) - s_masked) * m).sum()/m.sum()}"
            )
            if s is not None:
                print(f"iteration {i}, écart absolu moyen = {abs(s_masked-s).mean()}")
                print(f"iteration {i}, écart absolu moyen sur les données inconnues = {(abs(s_masked - s) * (1-m)).sum()/(1-m).sum()}")
                print(f"itération {i}, variance de la projection : {pca.explained_variance_ratio_}")
                print(f"itération {i}, modification moyenne des données inconnues {(abs(s_masked - old_s_masked)*(1-m)).sum()/(1-m).sum()}")
    recons_s = pca.inverse_transform(y)
    new_s_masked = recons_s * (1 - m) + s_masked * m
    return s_masked

def iter_tpca_score_estimation(
    s_masked,
    m,
    n_components=1,
    nb_iter=100,
    initial_noise_std=10,
    noise_std=0,
    verbose=False,
    initialisation="tensorial_product_estimation",
    s=None
):
    if initialisation == "tensorial_product_estimation":
        competition_vector, competitor_vector = estimate_tensorial_product(
            pd.DataFrame(s * m), participations_df=pd.DataFrame(m), removing_rate=0
        )
        s_estimated = competition_vector.values[:, np.newaxis] * competitor_vector.values[np.newaxis, :]
        s_masked = s_estimated * (1 - m) + s_masked * m
    elif initialisation == "random":
        s_masked = s_masked + np.random.randn(*s_masked.shape) * initial_noise_std * (1 - m) + (s_masked.sum() / m.sum()) * (1 - m)
    elif initialisation == "zeros":
        pass # s_masked = s_masked * m
    s_masked = s_masked.T
    m = m.T
    if s is not None:
        s = s.T
    for i in range(nb_iter):
        pca = PCA(n_components=n_components)
        y = pca.fit_transform(s_masked)
        recons_s = pca.inverse_transform(y)
        new_s_masked = (
            recons_s * (1 - m) + s_masked * m + np.random.randn(*s_masked.shape) * noise_std * (1 - m)
        )
        old_s_masked = s_masked
        s_masked = new_s_masked
        if i % max(nb_iter//10, 1) == 0 and verbose:
            print(
                f"\niteration {i}, écart absolu moyen sur la reconstruction des données connues = {(abs(pca.inverse_transform(y) - s_masked) * m).sum()/m.sum()}"
            )
            if s is not None:
                print(f"iteration {i}, écart absolu moyen = {abs(s_masked-s).mean()}")
                print(f"iteration {i}, écart absolu moyen sur les données inconnues = {(abs(s_masked - s) * (1-m)).sum()/(1-m).sum()}")
                print(f"itération {i}, variance de la projection : {pca.explained_variance_ratio_}")
                print(f"itération {i}, modification moyenne des données inconnues {(abs(s_masked - old_s_masked)*(1-m)).sum()/(1-m).sum()}")
    recons_s = pca.inverse_transform(y)
    new_s_masked = recons_s * (1 - m) + s_masked * m
    return s_masked.T

In [23]:
from collections import Counter

s, m, competitor_perfs, competition_length, s_no_noise, applied_coefs = make_fake_data2(
    3000, 300, 2000, noise_std=5, category_coef_std=0.03
)
# s, m, mu, _lambda = make_fake_data(3000, 300, 200, noise_std=0)
# s, m, mu, _lambda = make_fake_data(300, 30, 100, noise_std=0)
# s, m, mu, _lambda = make_fake_data(15, 5, 10, noise_std=0)
s_masked = s * m
s_masked = iter_tpca_score_estimation(
    s_masked,
    m,
    n_components=5,
    nb_iter=10,
    verbose=True,
    s=s,
    initialisation="tensorial_product_estimation",
)
print(f"\nErreur absolue moyenne : {abs(s_masked - s).mean()}")
print(
    f"Erreur absolue moyenne sur les valeurs reconstruites : {abs((s_masked - s) * (1-m)).sum()/(1-m).sum()}"
)
print(f"Erreur asolue maximale : {abs(s_masked - s).max()}")

counter_unseen = Counter(np.argmax(abs(s_masked - s_no_noise) * (1 - m), axis=1))
print(
    f"Indices les plus mal reconstruits parmis les données inconnues :\n{counter_unseen.most_common(10)}"
)
counter_seen = Counter(np.argmax(abs(s_masked - s_no_noise) * m, axis=1))
print(
    f"\nIndices les plus mal reconstruits parmis les données connues :\n{counter_seen.most_common(10)}"
)

print(
    f"\nNombre de competitions pour le compétiteur le plus mal reconstruit : {np.sum(m[:, counter_seen.most_common(1)[0][0]])}"
)
print(f"\nNombre de competitions moyen par competiteur : {np.sum(m, axis=0).mean()}")
j = 1
print(
    f"\nErreur max pour le {j}ème compétiteur le plus mal reconstruit : {abs(s_masked - s_no_noise)[:, counter_seen.most_common(j)[j-1][0]].max()}\n"
)
for i in range(20):
    print(
        f"Erreur sur la valeur min pour la course {i} : {abs(s_masked[i,:].min() - s_no_noise[i,:].min())}"
    )
print(
    f"\nErreur maximale de la valeur min sur l'ensemble des courses : {abs(s_masked.min(axis=1) - s_no_noise.min(axis=1)).max()}"
)
print(
    f"\nErreur moyenne de la valeur min sur l'ensemble des courses : {abs(s_masked.min(axis=1) - s_no_noise.min(axis=1)).mean()}"
)


Erreur moyenne due au bruit lors de la génération des données : 3.988898032074559

iteration 0, écart absolu moyen sur la reconstruction des données connues = 3.9929726823242193
iteration 0, écart absolu moyen = 1.3601007712756072
iteration 0, écart absolu moyen sur les données inconnues = 4.080302313826821
itération 0, variance de la projection : [8.86279796e-01 7.99208739e-03 4.66593124e-03 2.34488769e-03
 5.60764046e-04]
itération 0, modification moyenne des données inconnues 1.2999085468625258

iteration 1, écart absolu moyen sur la reconstruction des données connues = 3.9387325499669843
iteration 1, écart absolu moyen = 1.3482630236030824
iteration 1, écart absolu moyen sur les données inconnues = 4.044789070809247
itération 1, variance de la projection : [0.88280057 0.01384924 0.00804448 0.00404445 0.0008983 ]
itération 1, modification moyenne des données inconnues 0.4517816430561072

iteration 2, écart absolu moyen sur la reconstruction des données connues = 3.9312029747744233
i

## Utilisation du Bayesian principal component analysis (bpca)

(Temps de calcul trop long pour la taille de données souhaitée)

In [8]:
ignore = True
if not ignore:
    s, m, mu, _lambda = make_fake_data(3000, 300, 200, noise_std=5)
    #s, m, mu, _lambda = make_fake_data(300, 30, 100, noise_std=3)
    print(m.sum(axis=0).min())
    s_masked = s * m
    for i in range(s_masked.shape[0]):
        for j in range(s_masked.shape[1]):
            if m[i, j] == 0:
                s_masked[i, j] = np.nan
    s_masked = pd.DataFrame(s_masked)
    s_filled = bpca_fill(s_masked, max_iter=20, verbose=True)
    print(f"erreur absolue moyenne d'estimation des temps : {abs(s_filled-s).mean().mean()}")
    print(f"erreur moyenne d'estimation des temps : {(s_filled-s).mean().mean()}")

## Méthode par estimation des coefficients de competition et des coefficients de performance

In [15]:
#s, m, mu, _lambda = make_fake_data(300, 30, 100, noise_std=3)
#s, m, mu, _lambda = make_fake_data(3000, 300, 200, noise_std=3)
s, m, competitor_perfs, competition_length, s_no_noise, applied_coefs = make_fake_data2(3000, 300, 200, noise_std=5, category_coef_std=0.03)
competition_vector, competitor_vector = estimate_tensorial_product(
    pd.DataFrame(s * m), participations_df=pd.DataFrame(m), removing_rate=0
)
s_filled = competition_vector.values[:, np.newaxis] * competitor_vector.values[np.newaxis, :]
print(f"erreur absolue moyenne d'estimation des temps : {abs(s_filled-s).mean()}")
print(f"erreur d'estimation des temps : {(s_filled-s).mean()}")
print(
    f"Erreur absolue moyenne sur les valeurs reconstruites : {abs((s_filled - s) * (1-m)).sum()/(1-m).sum()}"
)
print(f"écart type de l'erreur d'estimation des temps : {(s_filled-s).std()}")
print(f"erreur absolue moyenne d'estimation des longueurs de course : {abs(competition_vector.values - competition_length).mean()}")
print(f"erreur absolue maximale d'estimation des participations : {abs(competitor_vector.values - competitor_perfs).max()}")
print(f"\nErreur max pour le {j}ème compétiteur le plus mal reconstruit : {abs(s_filled - s_no_noise)[:, counter_seen.most_common(j)[j-1][0]].max()}\n")
for i in range(20):
    print(f"Erreur sur la valeur min pour la course {i} : {abs(s_filled[i,:].min() - s_no_noise[i,:].min())}")
print(f"\nErreur maximale de la valeur min sur l'ensemble des courses : {abs(s_filled.min(axis=1) - s_no_noise.min(axis=1)).max()}")
print(f"\nErreur moyenne de la valeur min sur l'ensemble des courses : {abs(s_filled.min(axis=1) - s_no_noise.min(axis=1)).mean()}")

Erreur moyenne due au bruit lors de la génération des données : 3.9892826011004394
erreur absolue moyenne d'estimation des temps : 4.530270927065136
erreur d'estimation des temps : -0.00025303700794469553
Erreur absolue moyenne sur les valeurs reconstruites : 4.547384590621052
écart type de l'erreur d'estimation des temps : 5.689590798727802
erreur absolue moyenne d'estimation des longueurs de course : 0.013925015268163855
erreur absolue maximale d'estimation des participations : 26.40526666425481

Erreur max pour le 1ème compétiteur le plus mal reconstruit : 5.777238095458557

Erreur sur la valeur min pour la course 0 : 0.9986736406732888
Erreur sur la valeur min pour la course 1 : 0.25431412697564326
Erreur sur la valeur min pour la course 2 : 0.362328111872003
Erreur sur la valeur min pour la course 3 : 0.8509954999409501
Erreur sur la valeur min pour la course 4 : 0.09062116064083625
Erreur sur la valeur min pour la course 5 : 0.688856219317671
Erreur sur la valeur min pour la cour

## Calcul de points par estimation du classement parmis l'ensemble des compétiteurs

In [10]:
def compute_pts(s, category_part=[0.53, 0.23, 0.16, 0.08]):
    boundaries = [0] + [int(s.shape[1] * part) for part in np.cumsum(category_part)]
    pts = np.zeros(s.shape)
    for cat in range(len(boundaries)-1):
        sub_s = s[:, boundaries[cat]:boundaries[cat+1]]
        pts[:, boundaries[cat]:boundaries[cat+1]] = np.argsort(sub_s, axis=1)
    return pts

def evaluate_pts(s_estimated, s_true, category_part=[0.53, 0.23, 0.16, 0.08], max_ranking=500):
    pts_estimated = compute_pts(s_estimated, category_part)
    pts_true = compute_pts(s_true, category_part)
    pts_estimated[pts_estimated > max_ranking] = max_ranking
    pts_true[pts_true > max_ranking] = max_ranking
    return abs(pts_estimated-pts_true).mean()

## Comparaison des 2 méthodes pour l'estimation du rang au classement

In [29]:
pca_scores = list()
tprod_scores = list()
N_tests = 10
for i in range(N_tests):
    (
        s,
        m,
        competitor_perfs,
        competition_length,
        s_no_noise,
        applied_coefs,
    ) = make_fake_data2(
        3000,
        300,
        200,
        noise_std=3,
        category_coef_std=0.03,
    )
    s_masked = s * m


    s_estimated_pca = iter_tpca_score_estimation(
        s_masked.copy(),
        m,
        n_components=5,
        nb_iter=10,
        initial_noise_std=0,
        noise_std=0.0,
        verbose=False,
        s=s_no_noise,
    )
    competition_vector, competitor_vector = estimate_tensorial_product(
        pd.DataFrame(s_masked.copy()), participations_df=pd.DataFrame(m), removing_rate=0
    )
    s_estimated_tprod = (
        competition_vector.values[:, np.newaxis] * competitor_vector.values[np.newaxis, :]
    )
    pca_eval = evaluate_pts(s_estimated_pca, s)
    tprod_eval = evaluate_pts(s_estimated_tprod, s)
    pca_scores.append(pca_eval)
    tprod_scores.append(tprod_eval)

print(f"Moyenne des scores pour la méthode PCA : {np.mean(pca_scores)}")
print(f"Moyenne des scores pour la méthode Tensorial Product : {np.mean(tprod_scores)}")

Moyenne des scores pour la méthode PCA : 139.53579377777777
Moyenne des scores pour la méthode Tensorial Product : 139.54827555555556
